In [ ]:
import re
from collections import defaultdict
from datetime import datetime
import pandas as pd
from webeval.post_eval_analysis import aggregate_post_eval_errors, extract_last_error, extract_action_timing_stats
pd.set_option('display.max_rows', 500)

In [ ]:
from artifacts import EvalLogs
### path to eval logs
el = EvalLogs(az_folder = "/path/to/Fara/eval/runs/WebSurfer-fara-100-max_n_images-3/fara-7b/<username>/WebVoyager_WebVoyager_data_08312025.jsonl/<run-id>")
print(el)
el.mount()

# output_folder.map_folders
folders = el.map_folders(lambda traj: {'name': traj, 'files': list(traj.iterdir())}, 'traj')

/home/corbyrosset/anaconda3/envs/fara_webeval/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Local: /data/data/Fara/eval/runs/WebSurfer-fara-100-max_n_images-3/fara-7b/corbyrosset/WebVoyager_WebVoyager_data_08312025.jsonl/1


Processing folders in /data/data/Fara/eval/runs/WebSurfer-fara-100-max_n_images-3/fara-7b/corbyrosset/WebVoyager_WebVoyager_data_08312025.jsonl/1/traj...:   0%|          | 0/449 [00:00<?, ?it/s]

Processing folders in /data/data/Fara/eval/runs/WebSurfer-fara-100-max_n_images-3/fara-7b/corbyrosset/WebVoyager_WebVoyager_data_08312025.jsonl/1/traj...: 100%|██████████| 449/449 [00:00<00:00, 615.97it/s]


In [3]:


# Test the function with the examples
test_log1 = '''2025-08-06 18:10:11,637 [INFO] webeval.core.ulta_user_query_840 - [Execution ulta_user_query_840] Start
2025-08-06 18:10:11,685 [INFO] webeval.core.ulta_user_query_840 - [Execution ulta_user_query_840] Trajectory is not complete. Reexecuting...
2025-08-06 18:10:48,619 [INFO] webeval.core.ulta_user_query_840 - WebSurferEvent(source='WebSurfer', message='computer_use( {"action": "web_search", "query": "Nude Mood eyeshadow palette site:ulta.com", "thoughts": "To purchase the Nude Mood eyeshadow palette from Ulta, I need to locate the product on their website. Performing a web search will help me find the specific product page quickly."} )', url='https://www.google.com/', action='computer_use', arguments={'action': 'web_search', 'query': 'Nude Mood eyeshadow palette site:ulta.com', 'thoughts': 'To purchase the Nude Mood eyeshadow palette from Ulta, I need to locate the product on their website. Performing a web search will help me find the specific product page quickly.'})
2025-08-06 18:11:11,355 [ERROR] webeval.core.ulta_user_query_840 - Invalid action text: {"name": "computer_use", "arguments": {"action": "left_click", "coordinate": 136, 99]}}
2025-08-06 18:11:11,355 [ERROR] webeval.core.ulta_user_query_840 - Error parsing thoughts and action: Clicking on the first relevant link in the search results will likely lead directly to the product page for the Nude Mood eyeshadow palette on Ulta's website, allowing us to proceed with the purchase process.'''

error_type1, error_desc1 = extract_last_error(test_log1)
print(f"Test 1 - Error type: '{error_type1}', Description: '{error_desc1}'")

test_log2 = '''2025-08-06 20:22:51,954 [INFO] webeval.core.petsmart_user_query_2557 - [Execution petsmart_user_query_2557] Start
2025-08-06 20:23:34,363 [ERROR] webeval.core.petsmart_user_query_2557 - Web surfing error
Traceback (most recent call last):
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 150, in _recover_page
    await page.goto(current_url, timeout=timeout_secs * 1000)
  File "/home/corbyrosset/anaconda3/envs/spencers_webeval/lib/python3.12/site-packages/playwright/async_api/_generated.py", line 8992, in goto
    await self._impl_obj.goto(
  File "/home/corbyrosset/anaconda3/envs/spencers_webeval/lib/python3.12/site-packages/playwright/_impl/_page.py", line 556, in goto
    return await self._main_frame.goto(**locals_to_params(locals()))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/corbyrosset/anaconda3/envs/spencers_webeval/lib/python3.12/site-packages/playwright/_impl/_frame.py", line 153, in goto
    await self._channel.send(
  File "/home/corbyrosset/anaconda3/envs/spencers_webeval/lib/python3.12/site-packages/playwright/_impl/_connection.py", line 69, in send
    return await self._connection.wrap_api_call(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/corbyrosset/anaconda3/envs/spencers_webeval/lib/python3.12/site-packages/playwright/_impl/_connection.py", line 558, in wrap_api_call
    raise rewrite_error(error, f"{parsed_st['apiName']}: {error}") from None
playwright._impl._errors.TargetClosedError: Page.goto: Target page, context or browser has been closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 99, in wrapper
    await _recover_page(page, timeout_secs, logger)
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 156, in _recover_page
    raise Exception(f"playwright_controller._recover_page(): All recovery methods failed. Reload error: {e}, Goto error: {goto_error}")
Exception: playwright_controller._recover_page(): All recovery methods failed. Reload error: Page.reload: Target page, context or browser has been closed, Goto error: Page.goto: Target page, context or browser has been closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/orca_web_surfer/orca_qwen25vl_history_web_surfer.py", line 35, in _generate_reply
    request_halt, content = await self.__generate_reply(cancellation_token)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/orca_web_surfer/orca_qwen25vl_history_web_surfer.py", line 195, in __generate_reply
    result = await self._execute_tool(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/orca_web_surfer/orca_qwen25vl_web_surfer.py", line 190, in _execute_tool
    url=await self.playwright_controller.get_page_url(self.page),
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 105, in wrapper
    raise e from recovery_error
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 74, in wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 1000, in get_page_url
    await self._ensure_page_ready(page)
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 105, in wrapper
    raise e from recovery_error
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 74, in wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 354, in _ensure_page_ready
    await self.on_new_page(page)
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 105, in wrapper
    raise e from recovery_error
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 74, in wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/users/wangzhe/code/agento/agento/src/agento/agents/web_surfer/playwright_controller.py", line 338, in on_new_page
    await page.bring_to_front()
  File "/home/corbyrosset/anaconda3/envs/spencers_webeval/lib/python3.12/site-packages/playwright/async_api/_generated.py", line 9383, in bring_to_front
    return mapping.from_maybe_impl(await self._impl_obj.bring_to_front())
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/corbyrosset/anaconda3/envs/spencers_webeval/lib/python3.12/site-packages/playwright/_impl/_page.py", line 664, in bring_to_front
    await self._channel.send("bringToFront", None)
  File "/home/corbyrosset/anaconda3/envs/spencers_webeval/lib/python3.12/site-packages/playwright/_impl/_connection.py", line 69, in send
    return await self._connection.wrap_api_call(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/corbyrosset/anaconda3/envs/spencers_webeval/lib/python3.12/site-packages/playwright/_impl/_connection.py", line 558, in wrap_api_call
    raise rewrite_error(error, f"{parsed_st['apiName']}: {error}") from None
playwright._impl._errors.TargetClosedError: Page.bring_to_front: Target page, context or browser has been closed
2025-08-06 20:23:35,319 [INFO] webeval.core.petsmart_user_query_2557 - [Execution petsmart_user_query_2557] Completed'''

error_type2, error_desc2 = extract_last_error(test_log2)
print(f"Test 2 - Error type: '{error_type2}', Description: '{error_desc2}'")

# Test with an "Error running task" example
test_log3 = '''2025-08-06 20:23:35,319 [ERROR] webeval.core.nordstrom_user_query_9755 - [Execution nordstrom_user_query_9755] Error running task: some detailed error message'''

error_type3, error_desc3 = extract_last_error(test_log3)
print(f"Test 3 - Error type: '{error_type3}', Description: '{error_desc3}'")

# Test case for extracting average time between actions
# Test the timing extraction with the sample core.log provided
sample_log = '''2025-08-13 08:30:14,504 [INFO] webeval.core.costco_user_query_4323 - [Execution costco_user_query_4323] Start
2025-08-13 08:30:14,553 [INFO] webeval.core.costco_user_query_4323 - [Execution costco_user_query_4323] Trajectory is not complete. Reexecuting...
2025-08-13 08:30:15,995 [INFO] webeval.core.costco_user_query_4323 - Initializing BrowserBase session...
2025-08-13 08:30:19,919 [INFO] webeval.core.costco_user_query_4323 - Connected to BrowserBase session: https://browserbase.com/sessions/21ff0446-84cc-4083-85f8-72654b51c7ce
2025-08-13 08:30:40,031 [INFO] webeval.core.costco_user_query_4323 - WebSurferEvent(source='WebSurfer-SummarizedAction', message='Action 1: URL=https://www.google.com/, input_text({\'state_description\': \'google.com homepage showing search box focused\', \'reasoning\': "Search for the Zayne Leather Power Loveseat specifically on Costco\'s site", \'input_field_id\': 10, \'text_value\': \'Zayne Leather Power Loveseat site:costco.com\', \'press_enter\': True, \'delete_existing_text\': True})', url='https://www.google.com/', action=None, arguments=None)
2025-08-13 08:30:40,722 [INFO] webeval.core.costco_user_query_4323 - WebSurferEvent(source='WebSurfer', message='input_text( {"state_description": "google.com homepage showing search box focused", "reasoning": "Search for the Zayne Leather Power Loveseat specifically on Costco\'s site", "input_field_id": 10, "text_value": "Zayne Leather Power Loveseat site:costco.com", "press_enter": true, "delete_existing_text": true} )', url='https://www.google.com/', action='input_text', arguments={'state_description': 'google.com homepage showing search box focused', 'reasoning': "Search for the Zayne Leather Power Loveseat specifically on Costco's site", 'input_field_id': 10, 'text_value': 'Zayne Leather Power Loveseat site:costco.com', 'press_enter': True, 'delete_existing_text': True})
2025-08-13 08:31:23,685 [INFO] webeval.core.costco_user_query_4323 - WebSurferEvent(source='WebSurfer-SummarizedAction', message='Action 2: URL=https://www.google.com/search?q=Zayne+Leather+Power+Loveseat+site%3Acostco.com&form=QBLH&sp=-1&lq=0&pq=zayne+leather+power+loveseat+site%3Acostco.co&sc=10-43&qs=n&sk=&cvid=144A236AC9334CCA815928977A36C847, click({\'state_description\': "Bing search results page showing a Costco result for \'Zayne 2-piece Leather Power Reclining Set with …\'", \'reasoning\': \'To navigate to the Costco product page that includes the Zayne Leather Power Loveseat for purchase\', \'target_id\': 34})', url='https://www.google.com/search?q=Zayne+Leather+Power+Loveseat+site%3Acostco.com&form=QBLH&sp=-1&lq=0&pq=zayne+leather+power+loveseat+site%3Acostco.co&sc=10-43&qs=n&sk=&cvid=144A236AC9334CCA815928977A36C847', action=None, arguments=None)
2025-08-13 08:31:24,382 [INFO] webeval.core.costco_user_query_4323 - WebSurferEvent(source='WebSurfer', message='click( {"state_description": "Bing search results page showing a Costco result for \'Zayne 2-piece Leather Power Reclining Set with \\u2026\'", "reasoning": "To navigate to the Costco product page that includes the Zayne Leather Power Loveseat for purchase", "target_id": 34} )', url='https://www.google.com/search?q=Zayne+Leather+Power+Loveseat+site%3Acostco.com&form=QBLH&sp=-1&lq=0&pq=zayne+leather+power+loveseat+site%3Acostco.co&sc=10-43&qs=n&sk=&cvid=144A236AC9334CCA815928977A36C847', action='click', arguments={'state_description': "Bing search results page showing a Costco result for 'Zayne 2-piece Leather Power Reclining Set with …'", 'reasoning': 'To navigate to the Costco product page that includes the Zayne Leather Power Loveseat for purchase', 'target_id': 34})
2025-08-13 08:46:24,171 [INFO] webeval.core.costco_user_query_4323 - [Execution costco_user_query_4323] Completed'''

avg_delta, session_duration, num_actions = extract_action_timing_stats(sample_log)
print(f"Test 4 - Sample log analysis:")
print(f"\tNumber of WebSurfer actions: {num_actions}")
print(f"\tAverage time between actions: {avg_delta:.2f} seconds" if avg_delta else "Average time between actions: N/A (need at least 2 actions)")
print(f"\tSession duration: {session_duration:.2f} seconds ({session_duration/60:.2f} minutes)" if session_duration else "Session duration: N/A")
print(f"\tLong session (>30 min): {'Yes' if session_duration and session_duration > 1800 else 'No'}")

Test 1 - Error type: 'Error parsing thoughts and action', Description: 'Error parsing thoughts and action'
Test 2 - Error type: 'Web surfing error: playwright._impl._errors.TargetClosedError: Page.bring_to_front: Target page, context or browser has been closed', Description: 'Web surfing error: playwright._impl._errors.TargetClosedError: Page.bring_to_front: Target page, context or browser has been closed'
Test 3 - Error type: 'Error running task: some detailed error message', Description: 'Error running task: some detailed error message'
Test 4 - Sample log analysis:
	Number of WebSurfer actions: 2
	Average time between actions: 43.66 seconds
	Session duration: 969.67 seconds (16.16 minutes)
	Long session (>30 min): No


In [4]:
# Re-initialize error tracking dictionaries with the updated function
error_counts = defaultdict(int)
error_trajectories = defaultdict(list)
no_error_count = 0
total_trajectories = len(folders)

print(f"processing {total_trajectories} trajectories to extract errors...")

# Aggregate errors from the folders
df = aggregate_post_eval_errors(folders)

# Configure pandas display options for better formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)

# Display the DataFrame
df

processing 449 trajectories to extract errors...
Re-processing 449 trajectories with updated error extraction...
Processed 0/449 trajectories...
Processed 100/449 trajectories...
Processed 200/449 trajectories...
Processed 300/449 trajectories...
Processed 400/449 trajectories...
Completed processing 449 trajectories!
Trajectories with no errors: 219
Trajectories with errors: 230
Average score: 0.701 (from 351 trajectories with scores)

No detailed success metrics found (no 0.8-5-3.json files processed)
Final answer statistics:
  final_answer_file_not_found: 98
  From 351 trajectories with final_answer files:
    is_aborted == True: 0
    is_aborted == False: 351
    empty_answer_below_action_budget_but_not_aborted: 0

Token usage statistics:
  No token usage data found in final_answer files
A) Average time between actions: N/A (no sessions with multiple actions found)
B) Average session duration: -65697.96 ± 37333.07 seconds (-1094.97 ± 622.22 minutes) (across 351 sessions)
C) Average

,Error Type,Count,Percentage,Sample Trajectories
0,Page recovery failed,26,5.8%,"ArXiv--21, GitHub--9, ESPN--5, ... (+23 more)"
1,Error running task: Page.goto: Timeout 60000ms exceeded.,17,3.8%,"Google Map--20, Cambridge Dictionary--25, Cambridge Dictionary--27, ... (+14 more)"
2,"Error running task: Page.set_viewport_size: Target page, context or browser has been closed",11,2.4%,"ESPN--22, Amazon--13, GitHub--0, ... (+8 more)"
3,Error running task: Timeout 30000ms exceeded.,8,1.8%,"Booking--17, Cambridge Dictionary--38, Cambridge Dictionary--18, ... (+5 more)"
4,WARNING,7,1.6%,"Cambridge Dictionary--40, Cambridge Dictionary--34, Cambridge Dictionary--39, ... (+4 more)"
5,playwright_controller._recover_page(),5,1.1%,"BBC News--1, ArXiv--38, BBC News--20, ... (+2 more)"
6,Error running task: Page.screenshot: Timeout 15000ms exceeded.,5,1.1%,"Cambridge Dictionary--35, Apple--39, Booking--43, ... (+2 more)"
7,"Error running task: Page.evaluate: Execution context was destroyed, most likely because of a nav...",2,0.4%,"Booking--34, Booking--25"
8,[Execution GitHub--26] Max retries (5) reached. Task failed with stage=0.,1,0.2%,GitHub--26
9,[Execution ESPN--24] Max retries (5) reached. Task failed with stage=0.,1,0.2%,ESPN--24
